## Imports of dependencies

In [2]:
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd
import pandas as pd
from datasets import Dataset, load_dataset, load_from_disk
import optuna
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
import torch
from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset, load_dataset, load_from_disk
from transformers import AutoTokenizer
import wandb

[2025-06-18 22:52:56,832] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0618 22:52:58.902000 9888 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


## Loading dataset and prepare Dataframe to easy following process.

In [3]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'en')
dataset = dataset['train'].train_test_split(test_size=0.1)
df = pd.concat([dataset['train'].to_pandas(), dataset['test'].to_pandas()], ignore_index=True)
df = df.drop_duplicates(subset=['Question'])
print(df.head())
print(len(df))


                                            Question  \
0  What is the ideal management for a 40-year-old...   
1  A 54-year-old woman with a history of chronic ...   
2  A 27-year-old woman with mild persistent asthm...   
3  A 40-year-old man undergoing an insurance asse...   
4  A 43-year-old man presents with nasal congesti...   

                                         Complex_CoT  \
0  Alright, let's think about this case. We've go...   
1  Alright, so we have a 54-year-old woman who's ...   
2  Alright, so this woman with asthma is having t...   
3  Alright, we've got a 40-year-old guy who's goi...   
4  Alright, let's think about what's going on her...   

                                            Response  
0  The management of a 40-year-old female patient...  
1  The clinical presentation described—a chronic ...  
2  Based on the information provided, it is most ...  
3  The most likely diagnosis for the abnormal fin...  
4  Based on the symptoms and findings you've desc..

## Creating clusters to obtain significant subset of dataset

In [ ]:
texts = df["Question"].astype(str).tolist()

vectorizer = TfidfVectorizer(max_features=1024)
X = vectorizer.fit_transform(texts)

n_clusters = 1968
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(X)

closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
centroid_questions = [texts[i] for i in closest]

unique_centroid_questions = list(dict.fromkeys(centroid_questions))

centroids_df = pd.DataFrame({'centroid_question': unique_centroid_questions})
print(centroids_df.head())
print(f"Number of unique centroid questions: {len(centroids_df)}")

                                   centroid_question
0  During a laparotomy for an ovarian tumor, if a...
1  本例检查结果:Hh105g/L,WBC5.9×109/L,尿蛋白(+++),尿素氮20mmo...
2  A 9-month-old boy is brought to the physician ...
3  A 68-year-old man comes to the physician becau...
4  A female presents with symptoms and signs sugg...
Number of unique centroid questions: 1284


## Obtain the answers for the centroid questions

In [ ]:
centroids_with_responses = pd.merge(
    centroids_df,
    df[['Question', 'Response']],
    left_on='centroid_question',
    right_on='Question',
    how='left'
)[['Question', 'Response']]

centroids_with_responses = centroids_with_responses.drop_duplicates(subset=['Question'])

print(centroids_with_responses.head())
print(centroids_with_responses.shape)

                                            Question  \
0  A 78-year-old woman with a history of essentia...   
1  A 39-year-old woman presents with 5 days of pa...   
2  A child presents with an infective skin lesion...   
3  本例检查结果:Hh105g/L,WBC5.9×109/L,尿蛋白(+++),尿素氮20mmo...   
4  A 68-year-old woman comes to the physician for...   

                                            Response  
0  For managing a provoked deep venous thrombosis...  
1  Based on the information provided, the most li...  
2  To confirm the identity of the organism likely...  
3  The clinical staging of this patient aligns wi...  
4  The most likely underlying cause of this patie...  
(1250, 2)


## Save meaningful centroids dataset

In [ ]:
centroids_dataset = Dataset.from_pandas(centroids_with_responses, preserve_index=False)

centroids_dataset.save_to_disk("centroids_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1250 [00:00<?, ? examples/s]

## Load centroids dataset into memory

In [ ]:
loaded_dataset = load_from_disk("centroids_dataset")
print(loaded_dataset.to_pandas().head())
print(loaded_dataset.to_pandas().shape)
print(loaded_dataset)

                                            Question  \
0  A 78-year-old woman with a history of essentia...   
1  A 39-year-old woman presents with 5 days of pa...   
2  A child presents with an infective skin lesion...   
3  本例检查结果:Hh105g/L,WBC5.9×109/L,尿蛋白(+++),尿素氮20mmo...   
4  A 68-year-old woman comes to the physician for...   

                                            Response  
0  For managing a provoked deep venous thrombosis...  
1  Based on the information provided, the most li...  
2  To confirm the identity of the organism likely...  
3  The clinical staging of this patient aligns wi...  
4  The most likely underlying cause of this patie...  
(1250, 2)
Dataset({
    features: ['Question', 'Response'],
    num_rows: 1250
})


## Split centroids dataset into train and test subset

In [ ]:
loaded_dataset = loaded_dataset.train_test_split(test_size=0.1)
print(len(loaded_dataset["train"]))
print(len(loaded_dataset["test"]))
print(loaded_dataset["test"].to_pandas().head())

1125
125
                                            Question  \
0  In a 95% confidence interval for the prevalenc...   
1  According to the Factory Act of 1948, what is ...   
2  In a situation where a 32-year-old man with a ...   
3  CAD predisposing factors-a)  Homocysteinemiab)...   
4  A 4 year old child presents with acute watery ...   

                                            Response  
0  In a 95% confidence interval ranging from 56% ...  
1  According to the Factory Act of 1948, the maxi...  
2  Based on the symptoms described—coma, pinpoint...  
3  The risk factors for coronary artery disease (...  
4       The etiological agent is B. Giardia lamblia.  


## Select Hugging Face model and its tokenizer

In [ ]:
model_name = "nvidia/Nemotron-Research-Reasoning-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

## Function to tokenize and format the data

In [ ]:
def preprocess_function(examples):
    prompts = [f"[INST] {question} [/INST]" for question in examples['Question']]
    responses = examples['Response']
    
    texts = []
    for prompt, response in zip(prompts, responses):
        texts.append(f"{prompt} {response}")
    
    print(texts[:1])
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )
    
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

## All dataset is tokenized

In [ ]:
tokenized_dataset = loaded_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=loaded_dataset["train"].column_names
)

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

['[INST] A 7-month-old boy with an unremarkable past medical history is experiencing symptoms such as fever, chills, cough, runny nose, and watery eyes along with his elder brother. He has been diagnosed with an influenza virus infection, and this is his first exposure to the virus. What immune mechanism is most likely responsible for combating the influenza virus infection in this scenario? [/INST] In this scenario, since the 7-month-old boy is experiencing his first influenza virus infection, the innate immune response plays a pivotal role in combating the virus initially. The key elements of this response include the production of Type I interferons (such as IFN-alpha and IFN-beta) by virus-infected cells. These interferons act as signaling molecules, alerting neighboring cells to bolster their defenses against the viral infection and activating natural killer (NK) cells. NK cells then play an essential role in identifying and destroying infected cells, helping to control the spread

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

['[INST] In a 95% confidence interval for the prevalence of cancer among smokers aged over 65 years, ranging from 56% to 76%, what is the probability that the true prevalence is actually less than 56%? [/INST] In a 95% confidence interval ranging from 56% to 76% for the prevalence of cancer among smokers aged over 65, the probability that the true prevalence is actually less than 56% is 2.5%. This accounts for the lower tail of the distribution outside the confidence interval, with the remaining 2.5% being the probability that the true prevalence is more than 76%.']


## Create model shell from specified Hugging Face model's name, and using available GPU.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

## Grid Search of set of hyperparameters to find the best combination.

In [ ]:
def optuna_objective(trial):

    wandb.init(
        project="model-qwen-finetuned-medical-reasoning",
        name=f"trial_{trial.number}",
        reinit=True
    )
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16])
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [4, 8, 16])

    training_args = TrainingArguments(
        output_dir="./qwen-finetuned",
        run_name=f"trial_{trial.number}",
        overwrite_output_dir=True,
        num_train_epochs=5,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        save_steps=1000,
        save_total_limit=1,
        eval_steps=500,
        logging_dir="./logs",
        logging_steps=50,
        learning_rate=learning_rate,
        weight_decay=0.01,
        warmup_steps=200,
        lr_scheduler_type="cosine",
        report_to="wandb",
        fp16=False,
        bf16=True,
        gradient_checkpointing=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
    )

    trainer.train()
    eval_metrics = trainer.evaluate()
    wandb.finish()
    return eval_metrics["eval_loss"]


## Run Grid Search to find the best hypermarameter's combination

In [ ]:
# Run Optuna study
study = optuna.create_study(direction="minimize")  # minimize eval_loss
study.optimize(optuna_objective, n_trials=20)

print("Best trial:")
print(study.best_trial)
print("Best params:")
print(study.best_params)

[I 2025-06-09 09:17:00,709] A new study created in memory with name: no-name-9bda1e00-b368-4064-b036-3b0ada42bfc2
wandb: Currently logged in as: luis-orellana777 (luis-orellana777-sngular) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
eval/loss,0.93118
eval/runtime,2.3854
eval/samples_per_second,52.402
eval/steps_per_second,3.354
total_flos,2.579805945987072e+16


...

Step,Training Loss
50,0.152100
100,0.144400
150,0.138500


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▄▇██
train/global_step,▁▄▇██
train/grad_norm,▁██
train/learning_rate,▁▄█
train/loss,█▄▁
eval/loss,1.09559
eval/runtime,2.0668


[I 2025-06-09 11:12:00,913] Trial 19 finished with value: 1.0955885648727417 and parameters: {'learning_rate': 2.3904033239190128e-05, 'per_device_train_batch_size': 2, 'gradient_accumulation_steps': 16}. Best is trial 8 with value: 0.6598436832427979.


Best trial:
FrozenTrial(number=8, state=TrialState.COMPLETE, values=[0.6598436832427979], datetime_start=datetime.datetime(2025, 6, 9, 10, 4, 12, 537942), datetime_complete=datetime.datetime(2025, 6, 9, 10, 8, 54, 450388), params={'learning_rate': 2.6563467269626478e-05, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-05, step=None), 'per_device_train_batch_size': CategoricalDistribution(choices=(2, 4, 8, 16)), 'gradient_accumulation_steps': CategoricalDistribution(choices=(4, 8, 16))}, trial_id=8, value=None)
Best params:
{'learning_rate': 2.6563467269626478e-05, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 4}
